# Test notebook

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import folium
import streamlit as st
import os
import matplotlib
from rasterstats import zonal_stats
import rasterio
import requests

from src.config_parameters import params
from src.utils import *
from src.utils_population import *
from src.utils_plotting import *

2023-03-14 15:35:00.053 
  command:

    streamlit run /home/daniele/miniconda3/envs/proximity/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


## General parameters

In [3]:
verbose = True

## User input

In [4]:
# Parameters given by the user
# gdf_file = "test_data/mda_isochrones.geojson"
gdf_file = 'test_data/shakeintensity_complex.zip'
data_type = "constrained"
year = 2020
aggregated = True

## Process input data

In [5]:
# Create geopandas dataframes and prepare starting points
gdf, error = load_gdf(gdf_file)

if error:
    raise ValueError(error)

In [6]:
gdf

,id,grid_code,grid_value,mag,eventTime,updated,url,Shape__Are,Shape__Len,geometry
0,us6000jlqa,4,4.8,7.5,2023-02-06,2023-02-12,https://earthquake.usgs.gov/earthquakes/eventp...,7.495473e+11,7.218840e+07,"MULTIPOLYGON (((38.50207 32.83333, 38.48494 32..."
1,us6000jllz,4,4.8,7.8,2023-02-06,2023-02-12,https://earthquake.usgs.gov/earthquakes/eventp...,1.307015e+12,1.229430e+08,"MULTIPOLYGON (((35.95496 29.11667, 35.94378 29..."
2,us7000j56z,2,2.8,5.8,2023-01-18,2023-01-22,https://earthquake.usgs.gov/earthquakes/eventp...,9.668782e+10,3.006155e+07,"MULTIPOLYGON (((43.66047 36.58333, 43.65409 36..."
3,us7000j56z,3,3.8,5.8,2023-01-18,2023-01-22,https://earthquake.usgs.gov/earthquakes/eventp...,1.470852e+11,3.247177e+07,"MULTIPOLYGON (((44.24982 36.58351, 44.25000 36..."
4,us6000jk0t,2,2.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,5.373947e+10,2.705561e+07,"MULTIPOLYGON (((44.29176 36.60000, 44.29165 36..."
5,us6000jk0t,3,3.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,1.830746e+11,3.153923e+07,"MULTIPOLYGON (((45.10092 36.60000, 45.09923 36..."
6,us6000jk0t,4,4.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,2.170660e+10,5.601739e+06,"MULTIPOLYGON (((45.27513 37.43321, 45.27500 37..."
7,us6000jk0t,2,2.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,5.373947e+10,2.705561e+07,"MULTIPOLYGON (((44.29176 36.60000, 44.29165 36..."
8,us6000jk0t,3,3.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,1.830746e+11,3.153923e+07,"MULTIPOLYGON (((45.10092 36.60000, 45.09923 36..."
9,us6000jk0t,4,4.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,2.170660e+10,5.601739e+06,"MULTIPOLYGON (((45.27513 37.43321, 45.27500 37..."


## Run computation

In [10]:
%%time
# Add population data
# gdf = gdf.iloc[:5,]
gdf_with_pop = add_population_data(
    gdf=gdf,
    data_type=data_type,
    tif_folder='test_data/pop_data',
    data_folder='data',
    year=year,
    aggregated=aggregated,
    clobber=False,
    verbose=True,
    text_on_streamlit=False,
    progress_bar=False,
)

Finding intersections with countries...
Intersected countries: BGR, RUS, KWT, LBN, IRQ, GEO, GRC, ISR, ROU, SYR, EGY, AZE, TUR, ARM, IRN, UKR, JOR, SAU, CYP, PSE.
Country: BGR.
Aggregating raster...
Country: RUS.
Aggregating raster...
Country: KWT.
Aggregating raster...
Country: LBN.
Aggregating raster...
Country: IRQ.
Aggregating raster...
Country: GEO.
Aggregating raster...
Country: GRC.
Aggregating raster...
Country: ISR.
Aggregating raster...
Country: ROU.
Aggregating raster...
Country: SYR.
Aggregating raster...
Country: EGY.
Aggregating raster...
Country: AZE.
Aggregating raster...
Country: TUR.
Aggregating raster...
Country: ARM.
Aggregating raster...
Country: IRN.
Aggregating raster...
Country: UKR.
Aggregating raster...
Country: JOR.
Aggregating raster...
Country: SAU.
Aggregating raster...
Country: CYP.
Aggregating raster...
Country: PSE.
Aggregating raster...
Done!
CPU times: user 3min 14s, sys: 8.78 s, total: 3min 22s
Wall time: 3min 25s


In [11]:
gdf_with_pop

,id,grid_code,grid_value,mag,eventTime,updated,url,Shape__Are,Shape__Len,geometry,pop_tot
0,us6000jlqa,4,4.8,7.5,2023-02-06,2023-02-12,https://earthquake.usgs.gov/earthquakes/eventp...,7.495473e+11,7.218840e+07,"MULTIPOLYGON (((38.50207 32.83333, 38.48494 32...",37710290
1,us6000jllz,4,4.8,7.8,2023-02-06,2023-02-12,https://earthquake.usgs.gov/earthquakes/eventp...,1.307015e+12,1.229430e+08,"MULTIPOLYGON (((35.95496 29.11667, 35.94378 29...",95514312
2,us7000j56z,2,2.8,5.8,2023-01-18,2023-01-22,https://earthquake.usgs.gov/earthquakes/eventp...,9.668782e+10,3.006155e+07,"MULTIPOLYGON (((43.66047 36.58333, 43.65409 36...",3981709
3,us7000j56z,3,3.8,5.8,2023-01-18,2023-01-22,https://earthquake.usgs.gov/earthquakes/eventp...,1.470852e+11,3.247177e+07,"MULTIPOLYGON (((44.24982 36.58351, 44.25000 36...",9268054
4,us6000jk0t,2,2.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,5.373947e+10,2.705561e+07,"MULTIPOLYGON (((44.29176 36.60000, 44.29165 36...",2054801
5,us6000jk0t,3,3.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,1.830746e+11,3.153923e+07,"MULTIPOLYGON (((45.10092 36.60000, 45.09923 36...",11246746
6,us6000jk0t,4,4.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,2.170660e+10,5.601739e+06,"MULTIPOLYGON (((45.27513 37.43321, 45.27500 37...",589946
7,us6000jk0t,2,2.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,5.373947e+10,2.705561e+07,"MULTIPOLYGON (((44.29176 36.60000, 44.29165 36...",2054801
8,us6000jk0t,3,3.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,1.830746e+11,3.153923e+07,"MULTIPOLYGON (((45.10092 36.60000, 45.09923 36...",11246746
9,us6000jk0t,4,4.8,5.9,2023-01-28,2023-02-04,https://earthquake.usgs.gov/earthquakes/eventp...,2.170660e+10,5.601739e+06,"MULTIPOLYGON (((45.27513 37.43321, 45.27500 37...",589946


## Plot

In [13]:
aggregated=aggregated
joint=True
col_label = 'id'
legend_title = 'Admin level'
plot_title = 'Admin level'

fig = plot_pop_data(
    gdf=gdf_with_pop, 
    col_label=col_label,
    legend_title=legend_title, 
    plot_title=plot_title,
    aggregated=aggregated,
    joint=joint,
    cmap_name='viridis'
    )

fig.show()